# Introduction
In this Kaggle competition, survival predictive models were developed to improve the prediction of transplant survivial for patients undergoing allogeneic Hematopoietic Cell Transplantation (HCT)

# Installing Libraries and loading dataset

In [33]:
!pip install lifelines

In [92]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.utils import k_fold_cross_validation, concordance_index
from google.colab import drive
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [35]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/kaggle/train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Analyzing and Pre-processing Data

## Handling Categorical Variables
#### In this analysis, categorical variables have been handled using different encoding techniques depending on the presence of any inherent hierarchy or order within the data.

1. One-Hot Encoding: Applied when no natural hierarchy relationship existed.
2. Label Encoding: Applied when natural hierarchy relationship existed.

In [36]:
# Mapping Categories
def map_category(df):
    category_mapping = {
        'FKalone': 'FK-based treatments',
        'FK+ MMF +- others': 'FK-based treatments',
        'FK+ MTX +- others(not MMF)': 'FK-based treatments',
        'FK+- others(not MMF,MTX)': 'FK-based treatments',
        'CSA alone': 'CSA-based treatments',
        'CSA + MMF +- others(not FK)': 'CSA-based treatments',
        'CSA + MTX +- others(not MMF,FK)': 'CSA-based treatments',
        'CSA +- others(not FK,MMF,MTX)': 'CSA-based treatments',
        'Cyclophosphamide alone': 'Cyclophosphamide treatments',
        'Cyclophosphamide +- others': 'Cyclophosphamide treatments',
        'Other GVHD Prophylaxis': 'Other treatments',
        'TDEPLETION +- other': 'Other treatments',
        'TDEPLETION alone': 'Other treatments',
        'No GvHD Prophylaxis': 'No treatment',
        'CDselect alone': 'Other treatments',
        'CDselect +- other': 'Other treatments',
        'Parent Q = yes, but no agent': 'No treatment',
    }
    df['gvhd_proph'] = df['gvhd_proph'].replace(category_mapping)
    return df

# Mapping Columns with Low Cardinality
def map_low_cardinality(df):
    low_cardinality_3 = ["hepatic_mild", "arrhythmia", "obesity", "psych_disturb", "diabetes",
                         "renal_issue", "hepatic_severe", "cardiac", "prior_tumor", "peptic_ulcer",
                         "pulm_severe", "rheum_issue", "pulm_moderate"]
    low_cardinality_2 = ["vent_hist", "rituximab", "in_vivo_tcd"]

    for column in low_cardinality_3:
        df[column] = df[column].map({'No': -1, 'Yes': 1, 'Not done': 0})

    for column in low_cardinality_2:
        df[column] = df[column].map({'No': -1, 'Yes': 1, 'Not done': 0})

    return df

# Mapping Custom Columns
def map_custom_columns(df):
    custom_mappings = {
        "cmv_status": { '+/+': 1, '-/+': 0, '+/-': 0, '-/-': 1 },
        "conditioning_intensity": { 'MAC': 3, 'RIC': 2, "NMA": 1, "TBD": 0, "No drug reported": 0,
                                    "N/A, F(pre-TED) not submitted": 0 },
        "cyto_score_detail": { "Intermediate": 2, "Favorable": 3, "TBD": 0, "Not tested": 0, "Poor": 1 },
        "donor_related": { "Multiple donor (non-UCB)": 1, "Related": 1, "Unrelated": 0 },
        "sex_match": { "M-M": 1, "F-F": 1, "M-F": 0, "F-M": 0 },
    }

    for column, mapping in custom_mappings.items():
        df[column] = df[column].map(mapping)

    return df

# Encoding and Dummy Variables
def encode_columns(df):
    df['TBI_Performed'] = df['tbi_status'].apply(lambda x: 0 if x == 'No TBI' else 1)
    df.drop(columns=['tbi_status'], inplace=True)

    combinations = ['P/G', 'P/H', 'P/B', 'G/P', 'G/H', 'G/B', 'H/P', 'H/G', 'H/B', 'B/P', 'B/G', 'B/H']
    df['tce_imm_match'] = df['tce_imm_match'].apply(
        lambda value: 1 if value in ['P/P', 'G/G', 'H/H', 'B/B'] else 0 if value in combinations else value)

    return df

# Applying One-Hot Encoding (Get Dummies)
def apply_one_hot_encoding(df):
    category_cols = ['dri_score', 'cyto_score', 'prim_disease_hct', 'gvhd_proph', 'race_group']
    df = pd.get_dummies(df, columns=category_cols, drop_first=True)
    return df

# Dropping Unnecessary Columns
def drop_columns(df):
    df = df.copy()
    cols_to_drop = ['hla_match_c_high', 'hla_match_drb1_low', 'ethnicity', 'tce_match', 'mrd_hct', 'tce_div_match', 'ID']
    df.drop(columns=cols_to_drop, inplace=True, errors='ignore')
    return df

def map_custom_low_cardinality_2(df):
  custom_low_cardinality_2 = ["graft_type", "prod_type", "melphalan_dose"]
  for column in custom_low_cardinality_2:
      df[column] = df[column].map({ list(df[column].value_counts().index)[0]: +1, list(df[column].value_counts().index)[1]: -1})

  return df

# Full Preprocessing Pipeline
def preprocess_data(df):
    df = map_category(df)
    df = map_low_cardinality(df)
    df = map_custom_low_cardinality_2(df)
    df = map_custom_columns(df)
    df = encode_columns(df)
    df = apply_one_hot_encoding(df)  # This ensures categorical features are properly encoded
    df = drop_columns(df)
    return df



In [37]:
processed_data = preprocess_data(df)
processed_data.head()

,psych_disturb,diabetes,hla_high_res_8,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,hla_high_res_6,...,prim_disease_hct_Solid tumor,gvhd_proph_Cyclophosphamide treatments,gvhd_proph_FK-based treatments,gvhd_proph_No treatment,gvhd_proph_Other treatments,race_group_Asian,race_group_Black or African-American,race_group_More than one race,race_group_Native Hawaiian or other Pacific Islander,race_group_White
0,-1.0,-1.0,NaN,-1.0,6.0,-1,-1.0,-1.0,-1.0,6.0,...,False,False,True,False,False,False,False,True,False,False
1,-1.0,-1.0,8.0,-1.0,6.0,1,-1.0,-1.0,-1.0,6.0,...,False,False,False,False,True,True,False,False,False,False
2,-1.0,-1.0,8.0,-1.0,6.0,-1,-1.0,-1.0,-1.0,6.0,...,False,True,False,False,False,False,False,True,False,False
3,-1.0,-1.0,8.0,-1.0,6.0,-1,-1.0,-1.0,-1.0,6.0,...,False,False,True,False,False,False,False,False,False,True
4,-1.0,-1.0,8.0,-1.0,6.0,1,-1.0,-1.0,-1.0,6.0,...,False,False,False,False,True,False,False,False,False,False


## Handling Missing Values

  Used LightGMB to impute missing values

  1. Replaced categorical data with its mode
  2. Numerical data was replaced with its mean

In [38]:
def lightgbm_impute(df, column_name, cat_features=None, task_type='auto'):
    # Copy the dataframe to avoid modifying the original one
    df = df.copy()

    # Separate rows with and without missing values
    missing_mask = df[column_name].isnull()
    df_missing = df[missing_mask]
    df_not_missing = df[~missing_mask]

    # If no missing values, return the original DataFrame
    if df_missing.empty:
        return df

    # Determine the task type if not specified
    if task_type == 'auto':
        task_type = 'classification' if df[column_name].nunique() < 19 else 'regression'

    print(task_type)
    print(column_name)

    # Prepare features and target
    X = df_not_missing.drop(columns=[column_name])
    y = df_not_missing[column_name]
    X_missing = df_missing.drop(columns=[column_name])

    # Encode categorical features if any
    if cat_features:
        for cat in cat_features:
            X[cat] = X[cat].astype('category')
            X_missing[cat] = X_missing[cat].astype('category')

    # Split data for validation
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

    # Initialize LightGBM model
    model = (
        LGBMClassifier() if task_type == 'classification' else LGBMRegressor()
    )

    # Fit the model
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='logloss' if task_type == 'classification' else 'rmse',
        early_stopping_rounds=50,
        verbose=False
    )
    df.loc[missing_mask, column_name] = model.predict(X_missing)

    return df

In [39]:
def iterative_lightgbm_impute(df, cat_features=None, max_iter=5, verbose=True):
    df = df.copy()

    # Initialize missing values with mean/mode
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype == 'object' or df[col].dtype.name == 'category':
                df[col].fillna(df[col].mode()[0], inplace=True)
            else:
                df[col].fillna(df[col].mean(), inplace=True)

    for iteration in range(max_iter):
        if verbose:
            print(f"Iteration {iteration + 1}/{max_iter}")
        for column_name in df.columns:
            if df[column_name].isnull().any():
                if verbose:
                    print(f"  Imputing column: {column_name}")
                # Impute the current column
                df = lightgbm_impute(df, column_name, cat_features=cat_features)

    return df

In [40]:
processed_data = iterative_lightgbm_impute(processed_data)
processed_data.head()

<ipython-input-39-ca73a27e3237>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


Iteration 1/5
Iteration 2/5
Iteration 3/5
Iteration 4/5
Iteration 5/5


,psych_disturb,diabetes,hla_high_res_8,arrhythmia,hla_low_res_6,graft_type,vent_hist,renal_issue,pulm_severe,hla_high_res_6,...,prim_disease_hct_Solid tumor,gvhd_proph_Cyclophosphamide treatments,gvhd_proph_FK-based treatments,gvhd_proph_No treatment,gvhd_proph_Other treatments,race_group_Asian,race_group_Black or African-American,race_group_More than one race,race_group_Native Hawaiian or other Pacific Islander,race_group_White
0,-1.0,-1.0,6.876801,-1.0,6.0,-1,-1.0,-1.0,-1.0,6.0,...,False,False,True,False,False,False,False,True,False,False
1,-1.0,-1.0,8.000000,-1.0,6.0,1,-1.0,-1.0,-1.0,6.0,...,False,False,False,False,True,True,False,False,False,False
2,-1.0,-1.0,8.000000,-1.0,6.0,-1,-1.0,-1.0,-1.0,6.0,...,False,True,False,False,False,False,False,True,False,False
3,-1.0,-1.0,8.000000,-1.0,6.0,-1,-1.0,-1.0,-1.0,6.0,...,False,False,True,False,False,False,False,False,False,True
4,-1.0,-1.0,8.000000,-1.0,6.0,1,-1.0,-1.0,-1.0,6.0,...,False,False,False,False,True,False,False,False,False,False


## Addressing Multicolinearity
1. Calculated Variable Inflation Factor for all independent variables.
2. Removed variable with highest VIF and recalculate VIF for remaining independent variables
3. Repeated the process till all independent variables have VIF less than 4.

In [42]:
processed_data[processed_data.select_dtypes(include='bool').columns] = processed_data.select_dtypes(include='bool').astype(float)


In [43]:
non_numeric_cols = processed_data.select_dtypes(exclude=['number']).columns.tolist()
print(non_numeric_cols)

[]


In [44]:
# Replace cell 13 with this corrected code

vif_threshold = 4
X1 = processed_data.drop(columns=['efs', 'efs_time'], axis=1)  # Independent variables

# Convert all columns to numeric, coerce errors to NaN
X1 = X1.apply(pd.to_numeric, errors='coerce') # This line is crucial

vif_data = pd.DataFrame()
vif_data["feature"] = X1.columns
vif_data['VIF'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif_data = vif_data.sort_values(by='VIF', ascending=False)

while vif_data['VIF'].iloc[0] > vif_threshold:
    X1 = X1.drop(vif_data['feature'].iloc[0], axis=1)
    vif_data = pd.DataFrame()
    vif_data["feature"] = X1.columns
    vif_data['VIF'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
    vif_data = vif_data.sort_values(by='VIF', ascending=False)
vif_data.reset_index(drop=True, inplace=True)

vif_data

,feature,VIF
0,hepatic_mild,3.947703
1,dri_score_Intermediate,2.919249
2,donor_related,2.774774
3,prior_tumor,2.718110
4,cmv_status,2.562869
5,cyto_score_Intermediate,2.307099
6,cyto_score_Poor,2.280473
7,psych_disturb,2.206209
8,dri_score_N/A - pediatric,2.112498
9,comorbidity_score,1.990739


In [45]:
predictors = vif_data.feature.tolist()
predictors.extend(['efs', 'efs_time'])
outcome = df[['efs', 'efs_time']]
filtered_df = processed_data[predictors]

## Scaling

Scaled the values between 0 and 1 using MinMaxScaler

In [56]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_df = scaler.fit_transform(filtered_df)

scaled_df = pd.DataFrame(scaled_df, columns=filtered_df.columns)
scaled_df.head()

,hepatic_mild,dri_score_Intermediate,donor_related,prior_tumor,cmv_status,cyto_score_Intermediate,cyto_score_Poor,psych_disturb,dri_score_N/A - pediatric,comorbidity_score,...,prim_disease_hct_IMD,dri_score_Very high,gvhd_proph_No treatment,prim_disease_hct_Other acute leukemia,prim_disease_hct_HD,cyto_score_Not tested,prim_disease_hct_CML,dri_score_Missing disease status,efs,efs_time
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.268542
1,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.027728
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.124356
3,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651918
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101543


# Predictive Analysis

## XGBoost Predictive Model

1. Used KaplanMeierFitter to fit dependend variables 'efs' and 'efs_time' into single y variable.
2. Used XGBRegressor with parameters:
n_estimators=100, learning_rate=0.1, max_depth=6, objective="reg:squarederror".
Split dataset into 90% train / 10% test.
Predictions & Evaluation
3. Trained model to predict survival probabilities.
Evaluated using Concordance Index (C-Index).

In [103]:

xgb_regressor = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    objective="reg:squarederror"
)

In [104]:
def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    kmf = KaplanMeierFitter()
    kmf.fit(df[time_col], df[event_col])
    y = kmf.survival_function_at_times(df[time_col]).values
    return y

In [105]:
xgboost_df = scaled_df.copy()

xgboost_df["y"] = transform_survival_probability(xgboost_df, time_col='efs_time', event_col='efs')
xgboost_df.drop(columns=['efs_time', 'efs'], axis=1, inplace=True)

def train_model_xgb(df_raw):
    df_raw_copy = df_raw.copy()

    y = df_raw_copy['y']
    df_raw_copy.drop(columns=['y'], axis=1, inplace=True)
    X =  df_raw_copy.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    xgb_regressor.fit(X_train, y_train)

    return xgb_regressor, X_test, y_test

    # y_pred = xgb_regressor.predict(X_test)

xgb_regressor, X_test_xgb, y_test_xgb = train_model_xgb(xgboost_df)


In [106]:
ypredxgb = xgb_regressor.predict(X_test_xgb)

In [107]:
y_test_xgb.values

array([0.46056945, 0.64258774, 0.69024306, ..., 0.68551659, 0.45795546,
       0.45977379])

In [119]:
X_test_xgb.shape

(7200, 53)

In [120]:
ypredxgb.shape

(7200,)

In [109]:
mse = mean_squared_error(y_test_xgb, ypredxgb)
print("mse ", mse)
print("r2_score ", r2_score(y_test_xgb, ypredxgb))

mse  0.027540262581501163
r2_score  0.11943892361842123


## Survival Analysis using Cox Proportional Hazard Model
This model is used for analyzing and modeling survival data mainly in medical field to predict survival probability of patient and in engineering field to predict survival time of parts.

Lifelines library was used to build the model

1. Duration of hazard was represented by column 'efs_time'
2. hazard event was represented by column 'efs'
3. K-fold cross valdiation was used to find optimal penalizer for the Cox Model
4. Independent variables having hazard ratio 1 were removed to improve the model performance

In [110]:
features = scaled_df.drop(columns=['efs', 'efs_time'])
event_column = 'efs'
X_train, X_test, y_train, y_test = train_test_split(scaled_df, scaled_df[['efs_time', 'efs']], test_size=0.25, random_state=42)

In [111]:
# List of different penalizer values (Lasso regularization strengths)
penalizer_values = [0.001, 0.01, 0.1, 1, 10]

# Store the mean log-likelihood for each penalizer value
mean_log_likelihoods = []

# Perform k-fold cross-validation for each penalizer value
for penalizer in penalizer_values:
    cph = CoxPHFitter(penalizer=penalizer)  # Set penalizer (Lasso regularization strength)
    # Pass the complete DataFrame (scaled_df) to k_fold_cross_validation
    # and specify the duration and event columns
    cross_val_results = k_fold_cross_validation(cph, scaled_df, duration_col='efs_time', event_col='efs', k=5)

    # Calculate the mean log-likelihood for the current penalizer value
    mean_log_likelihood = np.mean(cross_val_results)
    mean_log_likelihoods.append(mean_log_likelihood)

    print(f"Penalizer: {penalizer}, Mean Log-Likelihood: {mean_log_likelihood}")

# Find the penalizer value with the best mean log-likelihood
best_penalizer = penalizer_values[np.argmax(mean_log_likelihoods)]
print(f"The best penalizer value is: {best_penalizer}")

Penalizer: 0.001, Mean Log-Likelihood: -4.410502938035536
Penalizer: 0.01, Mean Log-Likelihood: -4.410372664168283
Penalizer: 0.1, Mean Log-Likelihood: -4.4126352718868755
Penalizer: 1, Mean Log-Likelihood: -4.436243898856027
Penalizer: 10, Mean Log-Likelihood: -4.472732617435876
The best penalizer value is: 0.01


In [112]:
cph = CoxPHFitter(penalizer=best_penalizer)
cph.fit(X_train, 'efs_time', 'efs', robust=True)
cph.print_summary()

model,lifelines.CoxPHFitter
duration col,'efs_time'
event col,'efs'
penalizer,0.01
l1 ratio,0.0
robust variance,True
baseline estimation,breslow
number of observations,21600
number of events observed,11671
partial log-likelihood,-110847.65
time fit was run,2025-01-05 22:21:57 UTC


In [113]:
import pandas as pd

# Assuming 'filtered_data' is your DataFrame containing the predictors

# List of predictors you want to remove
columns_to_remove = [
    'dri_score_N/A - pediatric',
    'dri_score_N/A - non-malignant indication',
    'tce_match_GvH non-permissive',
    'dri_score_TBD cytogenetics',
    'prior_tumor',
    'prim_disease_hct_NHL',
    'tbi_status_TBI +- Other, >cGy',
    'dri_score_High - TED AML case <missing cytogenetics',
    'cyto_score_detail_Favorable',
    'obesity',
    'arrhythmia',
    'cyto_score_Normal',
    'prim_disease_hct_AI',
    'dri_score_N/A - disease not classifiable',
    'donor_related_Multiple donor (non-UCB)',
    'tbi_status_TBI +- Other, -cGy, single',
    'tbi_status_TBI +- Other, -cGy, fractionated',
    'tbi_status_TBI +- Other, unknown dose',
    'conditioning_intensity_N/A, F(pre-TED) not submitted',
    'tbi_status_TBI +- Other, unknown dose',
    'cyto_score_Not tested',
    'prim_disease_hct_CML'
]

# Remove the specified columns from your DataFrame
selected_df = X_train.drop(columns=columns_to_remove, errors='ignore')

# Display the updated DataFrame (without the specified columns)
selected_df.head()


,hepatic_mild,dri_score_Intermediate,donor_related,cmv_status,cyto_score_Intermediate,cyto_score_Poor,psych_disturb,comorbidity_score,diabetes,sex_match,...,cyto_score_Other,prim_disease_hct_Solid tumor,prim_disease_hct_IMD,dri_score_Very high,gvhd_proph_No treatment,prim_disease_hct_Other acute leukemia,prim_disease_hct_HD,dri_score_Missing disease status,efs,efs_time
16238,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.4,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.024948
26642,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.2,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.058012
5504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.238219
27453,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.4,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.085068
766,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.3,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.044419


In [114]:
cph = CoxPHFitter(penalizer=best_penalizer)
cph.fit(selected_df, 'efs_time', 'efs', robust=True)
cph.print_summary()

model,lifelines.CoxPHFitter
duration col,'efs_time'
event col,'efs'
penalizer,0.01
l1 ratio,0.0
robust variance,True
baseline estimation,breslow
number of observations,21600
number of events observed,11671
partial log-likelihood,-110971.60
time fit was run,2025-01-05 22:23:13 UTC


In [115]:
c_index = cph.concordance_index_
print(f"Concordance Index (C-index): {c_index}")

Concordance Index (C-index): 0.6398748216171377


## Combination of XGBoost and Cox-Proportional Hazard model using concordence index

In [116]:
def return_concordance_index(event_times, event_observed, predicted_scores):
    c_index = concordance_index(event_times, predicted_scores, event_observed)
    return c_index

In [121]:
X_test["efs_time"].shape

(7200,)

In [122]:
y_pred_xgb.shape

(2880,)

In [123]:
y_pred_xgb = xgb_regressor.predict(X_test_xgb)
# Use predict_partial_hazard instead of predict
y_pred_cox = cph.predict_partial_hazard(X_test)


concordance_xgb = return_concordance_index(X_test["efs_time"], X_test["efs"], y_pred_xgb)
concordance_cox = return_concordance_index(X_test["efs_time"], X_test["efs"], y_pred_cox)

y_pred = (concordance_xgb * y_pred_xgb + concordance_cox * y_pred_cox) / (concordance_xgb + concordance_cox)

y_pred

,0
18932,1.304591
21280,0.958290
27880,0.918086
15692,1.158744
25416,1.068040
...,...
18078,1.023982
3566,0.984713
4129,0.679819
9939,0.753674


In [124]:
print("concordance_xgb", concordance_xgb)
print("concordance_cox", concordance_cox)

concordance_final = return_concordance_index(X_test["efs_time"], X_test["efs"], y_pred)
print("concordance_final", concordance_final)

concordance_xgb 0.36246053274699186
concordance_cox 0.3620423046724926
concordance_final 0.3592257521938968


# Conclusion
The results indicate similar performance across XGBoost, Cox, and ensemble models, with no significant gain from ensembling.

Key insights from feature importance analysis suggest that comorbidities, disease type, and cytogenetic risk scores play a crucial role in survival prediction.

## Top 5 Features Increasing Risk (Hazard Ratio > 1)
1. comorbidity_score → 2.21
2. prod_type → 1.42
3. cyto_score_Poor → 1.22
4. prim_disease_hct_AML → 1.05
5. prim_disease_hct_IEA → 1.08
## Top 5 Features Decreasing Risk (Hazard Ratio < 1)
1. prim_disease_hct_HD → 0.15
2. hepatic_mild → 0.95
3. dri_score_Intermediate → 0.82
4. gvhd_proph_Cyclophosphamide treatments → 0.81
5. race_group_More than one race → 0.90